# Federated K-Means Clustering with Scikit-learn on Iris Dataset

## Introduction to Scikit-learn, tabular data, and federated k-Means
### Scikit-learn
This example shows how to use [NVIDIA FLARE](https://nvflare.readthedocs.io/en/main/index.html) on tabular data.
It uses [Scikit-learn](https://scikit-learn.org/),
a widely used open-source machine learning library that supports supervised 
and unsupervised learning.
### Tabular data
The data used in this example is tabular in a format that can be handled by [pandas](https://pandas.pydata.org/), such that:
- rows correspond to data samples
- the first column represents the label 
- the other columns cover the features.    

Each client is expected to have one local data file containing both training 
and validation samples. To load the data for each client, the following 
parameters are expected by the local learner:
- data_file_path: string, the full path to the client's data file 
- train_start: int, start row index for the training set
- train_end: int, end row index for the training set
- valid_start: int, start row index for the validation set
- valid_end: int, end row index for the validation set

### Federated k-Means clustering
The machine learning algorithm in this example is [k-Means clustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).
The aggregation follows the scheme defined in [Mini-batch k-Means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html). 
Under this setting, each round of federated learning can be formulated as follows:
- local training: starting from global centers, each client trains a local MiniBatchKMeans model with their own data
- global aggregation: server collects the cluster center, 
  counts information from all clients, aggregates them by considering 
  each client's results as a mini-batch, and updates the global center and per-center counts.

For center initialization, at the first round, each client generates its initial centers with the k-means++ method. Then, the server collects all initial centers and performs one round of k-means to generate the initial global center.

Below we listed steps to run this example.

## 1. Setup NVFLARE

Follow the [Getting_Started](https://nvflare.readthedocs.io/en/main/getting_started.html) to setup virtual environment and install NVFLARE

We also provide a [Notebook](../../nvflare_setup.ipynb) for this setup process. 

Assume you have already setup the venv, lets first install required packages.

In [ ]:
%pip install -r requirements.txt

## 2. Data preparation 
This example uses the Iris dataset available from Scikit-learn's dataset API.  

In [6]:
%env DATASET_PATH=/tmp/nvflare/dataset/sklearn_iris.csv
!python3 ./utils/prepare_data.py --dataset_name iris --out_path ${DATASET_PATH}

env: DATASET_PATH=/tmp/nvflare/dataset/sklearn_iris.csv


## 4. Run simulated kmeans experiment
We run the federated training using NVFlare Simulator via [JobAPI](https://nvflare.readthedocs.io/en/main/programming_guide/fed_job_api.html):

In [ ]:
! python kmeans_job.py --num_clients 3 --split_mode uniform

job_dir= /tmp/nvflare/workspace/jobs/kmeans
workspace_dir= /tmp/nvflare/workspace/works/kmeans/sklearn_kmeans_uniform_3_clients
2025-02-10 17:57:48,387 - SimulatorRunner - INFO - Create the Simulator Server.
2025-02-10 17:57:48,388 - CoreCell - INFO - server: creating listener on tcp://0:56259
2025-02-10 17:57:48,406 - CoreCell - INFO - server: created backbone external listener for tcp://0:56259
2025-02-10 17:57:48,406 - ConnectorManager - INFO - 13968: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2025-02-10 17:57:48,407 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:28310] is starting
2025-02-10 17:57:48,908 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:28310
2025-02-10 17:57:48,909 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:56259] is starting
2025-02-10 17:57:48,911 - SimulatorServer - INFO - max_reg_duration=60.0
2025-02-10 17:57:49,003 - nvflare

## 5. Result visualization
Model accuracy is computed as the homogeneity score between the cluster formed and the ground truth label, which can be visualized in tensorboard.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /tmp/nvflare/workspace/works/kmeans/sklearn_kmeans_uniform_3_clients